# AI for Cybersecurity: Simulating and Analyzing Cyber Attack Paths

**Copyright (c) 2026 Shrikara Kaudambady. All rights reserved.**

This notebook demonstrates how to model a computer network as a graph and use AI pathfinding algorithms to simulate and analyze potential cyber attack paths. By thinking like an attacker, security teams can identify and prioritize the most critical vulnerabilities to fix.

### 1. Setup and Library Imports

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

### 2. Simulate the Corporate Network Environment

We'll define our network as a list of nodes (servers) and edges (connections). Each server has properties like its value to an attacker and a list of known vulnerabilities.

In [ ]:
nodes_data = {
    'WebServer': {'value': 20, 'vulnerabilities': ['Apache-Log4j']},
    'AppServer': {'value': 50, 'vulnerabilities': ['Java-RCE', 'UnpatchedOS']},
    'Database': {'value': 90, 'vulnerabilities': []},
    'DevMachine': {'value': 10, 'vulnerabilities': ['OpenSSH-CVE']},
    'DomainController': {'value': 100, 'vulnerabilities': ['ZeroLogon']},
    'UserPC': {'value': 5, 'vulnerabilities': ['UnpatchedOS']}
}

# Edges represent network connectivity
edges_data = [
    ('WebServer', 'AppServer'),
    ('WebServer', 'UserPC'),
    ('AppServer', 'Database'),
    ('UserPC', 'DevMachine'),
    ('DevMachine', 'AppServer'),
    ('DevMachine', 'DomainController'),
    ('AppServer', 'DomainController')
]

print("Network environment defined.")

### 3. Build the Network Graph

We create a function to build a `networkx` graph from our environment data. The core of the simulation is defining the 'cost' for an attacker to move between nodes. We will model this cost as being inversely related to the number of vulnerabilities on the target node.

In [ ]:
def build_attack_graph(nodes, edges, patched_vulns=[]):
    G = nx.Graph()
    
    # Add nodes with their attributes
    for node_id, attrs in nodes.items():
        G.add_node(node_id, **attrs)
    
    # Add edges with a calculated 'cost' for traversal
    base_cost = 10
    vulnerability_discount = 4
    
    for u, v in edges:
        # Cost is determined by the destination node's vulnerabilities
        target_node_vulns = set(nodes[v]['vulnerabilities']) - set(patched_vulns)
        cost = base_cost - (len(target_node_vulns) * vulnerability_discount)
        G.add_edge(u, v, weight=max(1, cost)) # Ensure cost is at least 1
        
    return G

def visualize_graph(G, path=None):
    pos = nx.spring_layout(G, seed=42)
    
    # Node colors based on value
    node_colors = [G.nodes[n]['value'] for n in G.nodes]
    
    plt.figure(figsize=(12, 8))
    nx.draw(G, pos, with_labels=True, node_color=node_colors, cmap=plt.cm.YlOrRd, node_size=2500, font_size=10, font_weight='bold')
    
    # Highlight the attack path if provided
    if path:
        path_edges = list(zip(path, path[1:]))
        nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='red', width=2)
    
    plt.title("Simulated Corporate Network")
    plt.show()

### 4. Simulation 1: Baseline Attack Path

First, we'll find the attacker's path of least resistance in the original, unpatched network. The goal is to get from the public-facing 'WebServer' to the 'DomainController'.

In [ ]:
print("--- SIMULATION 1: BASELINE ---")
G_baseline = build_attack_graph(nodes_data, edges_data)

start_node = 'WebServer'
end_node = 'DomainController'

# Use Dijkstra's algorithm to find the lowest-cost path
baseline_path = nx.dijkstra_path(G_baseline, start_node, end_node, weight='weight')
baseline_cost = nx.dijkstra_path_length(G_baseline, start_node, end_node, weight='weight')

print(f"Initial Attack Path: {' -> '.join(baseline_path)}")
print(f"Total Attack Cost: {baseline_cost}")

visualize_graph(G_baseline, path=baseline_path)

### 5. Simulation 2: Post-Remediation Attack Path

The simulation shows the easiest path is via the 'DevMachine'. Let's say our security team patches the 'OpenSSH-CVE' vulnerability on that machine. How does this change the attacker's strategy?

In [ ]:
print("\n--- SIMULATION 2: AFTER PATCHING 'DevMachine' ---")
patched_vulnerabilities = ['OpenSSH-CVE']

G_patched = build_attack_graph(nodes_data, edges_data, patched_vulns=patched_vulnerabilities)

# Find the new optimal path
patched_path = nx.dijkstra_path(G_patched, start_node, end_node, weight='weight')
patched_cost = nx.dijkstra_path_length(G_patched, start_node, end_node, weight='weight')

print(f"New Attack Path: {' -> '.join(patched_path)}")
print(f"New Total Attack Cost: {patched_cost}")

visualize_graph(G_patched, path=patched_path)

### 6. Analysis and Conclusion

By patching the vulnerability on the `DevMachine`, we made that route more 'costly' for the attacker. The simulation shows that the AI pathfinding logic now chooses a different, more expensive route through the `AppServer`.

This type of simulation is incredibly valuable for security teams. It allows them to:
- **Prioritize Vulnerabilities:** Instead of patching all vulnerabilities equally, teams can focus on those that lie on the most critical attack paths.
- **Quantify Risk Reduction:** We can assign a numerical value to our security posture by measuring the cost of the most likely attack paths.
- **Test Defenses:** Before deploying a new firewall rule or security control, it can be modeled in the graph to see how effectively it disrupts potential attackers.